In [1]:
#####################
###### Imports ######
#####################

import os
from configparser import ConfigParser
import sys
import re
if not os.getcwd().endswith('trading'): os.chdir('../../..') # local machine
assert os.getcwd().endswith('trading'), 'Wrong path!'
import numerapi
import plotly.express as px


os.environ['NUMEXPR_MAX_THREADS'] = '32'
os.environ['NUMEXPR_NUM_THREADS'] = '16'

sys.path.append(os.getcwd())
from dev.scripts.ML_utils import * # run if on local machine
from dev.scripts.trading_utils import * # run if on local machine
from numerai.dev.scripts.numerai_utils import *
from numerai.dev.configs.prep_and_train_cfg import *

pd.set_option('display.float_format', lambda x: '%.5f' % x)
config = ConfigParser()
config.read('numerai/numerai_keys.ini')is_f

# Connect to the Numerai API
napi = numerapi.SignalsAPI(config['KEYS']['NUMERAI_PUBLIC_KEY'], config['KEYS']['NUMERAI_SECRET_KEY'])

### Load in the data created from build_numerai_dataset.py ###

if LOAD_DATA_FILEPATH.endswith('feather'):
    df_numerai = pd.read_feather(LOAD_DATA_FILEPATH)
elif LOAD_DATA_FILEPATH.endswith('pq') or LOAD_DATA_FILEPATH.endswith('parquet'):
    df_numerai = pd.read_parquet(LOAD_DATA_FILEPATH)
elif LOAD_DATA_FILEPATH.endswith('csv'):
    df_numerai = pd.read_csv(LOAD_DATA_FILEPATH)

In [2]:
### dropnas ###

if DROP_NA_TARGETS:
    df_numerai.dropna(subset=[TARGET], how='any', inplace=True)
    df_numerai.reset_index(inplace=True)


### Timeseries Split ###

df_numerai = timeseries_split(df_numerai, **TIMESERIES_SPLIT_PARAMS)

if START_DATE:
    df_numerai = df_numerai[df_numerai[DATE_COL] >= START_DATE]


### train test split ###

input_features = eval(INPUT_FEATURES_STRING)
preserve_vars = eval(PRESERVE_VARS_STRING)

X_train, y_train = df_numerai[df_numerai[SPLIT_COLNAME].str.startswith('train')][input_features], df_numerai[df_numerai[SPLIT_COLNAME].str.startswith('train')][TARGET]
X_val, y_val = df_numerai[df_numerai[SPLIT_COLNAME].str.startswith('val')][input_features], df_numerai[df_numerai[SPLIT_COLNAME].str.startswith('val')][TARGET]
X_test, y_test = df_numerai[df_numerai[SPLIT_COLNAME].str.startswith('test')][input_features], df_numerai[df_numerai[SPLIT_COLNAME].str.startswith('test')][TARGET]

In [3]:
### Preprocessing ###

# df_train = df_numerai[list(set(input_features + preserve_vars + [TARGET]))]

PREPROCESS_FEATURES_PARAMS['preserve_vars'] = preserve_vars
feature_transformer = PreprocessFeatures(**PREPROCESS_FEATURES_PARAMS).fit(X_train, y_train)

Overlap between numeric and oh_features: ['prev2_target_HL5', 'prev5_target', 'prev1_target_HL5', 'prev4_target_HL3', 'prev1_target_HL3', 'prev4_target', 'prev2_target_HL3', 'prev5_target_HL3', 'prev3_target_HL3', 'prev1_target', 'prev5_target_HL5', 'prev3_target', 'prev4_target_HL5', 'prev2_target', 'prev3_target_HL5']
Overlap between numeric and hc_features: []
Overlap between numeric oh_features and hc_features: []
Overlap between oh_features and hc_features will be moved to oh_features

numeric_features:['high_move_pct_1d_diff', 'high_minus_close_1d_diff_pct_change', 'high_minus_prev_low_1d_pct_change', 'prev_close_minus_low_1d_pct_change', 'move_1d_iar', 'high_move_1d', 'open_minus_prev_close_1d_diff_pct_change', 'prev_close_pct_chg_1d_diff', 'prev4_volume_1d', 'low_move_1d_diff', 'high_move_pct_1d_diff_pct_change', 'prev_close_minus_low_1d', 'open_minus_prev_close_1d_pct_change', 'prev1_target_HL5_ewm_mean', 'prev1_target_rolling_mean', 'prev4_move_1d', 'move_1d_diff_pct_change',

In [68]:
FEATURE_TRANSFORMER_OUTPATH + 'feature_transformer_' + str(datetime.datetime.today().replace(second=0, microsecond=0)).replace(' ', '_').replace(':', '_')

'/media/melgazar9/HDD_10TB/trading/objects/feature_transformers/feature_transformer_2021-04-24_13_46_00'

In [73]:
FEATURE_TRANSFORMER_OUTPATH == '/media/melgazar9/HDD_10TB/trading/objects/feature_transformers/'

True

In [75]:
if SAVE_FEATURE_TRANSFORMER:
    dill.dump(feature_transformer, open(FEATURE_TRANSFORMER_OUTPATH + 'feature_transformer_' + str(datetime.datetime.today().replace(second=0, microsecond=0)).replace(' ', '_').replace(':', '_') + '.pkl', 'wb'))

In [4]:
final_features = FeatureImportance(feature_transformer, verbose=VERBOSE).get_feature_names()
X_train_transformed = pd.DataFrame(feature_transformer.transform(X_train), columns=final_features)
X_val_transformed = pd.DataFrame(feature_transformer.transform(X_val), columns=final_features)
X_test_transformed = pd.DataFrame(feature_transformer.transform(X_test), columns=final_features)
X_transformed = pd.concat([X_train_transformed, X_val_transformed, X_test_transformed])



---------
Running get_feature_names
---------



 0 . Transformer/Pipeline:  hc_pipeline , Pipeline 

	n_orig_feature_names: 0
transformer 0 is not fitted!


 1 . Transformer/Pipeline:  numeric_pipeline , Pipeline 

	n_orig_feature_names: 95
	n_new_features: 190
	new_features:
 ['high_move_pct_1d_diff', 'high_minus_close_1d_diff_pct_change', 'high_minus_prev_low_1d_pct_change', 'prev_close_minus_low_1d_pct_change', 'move_1d_iar', 'high_move_1d', 'open_minus_prev_close_1d_diff_pct_change', 'prev_close_pct_chg_1d_diff', 'prev4_volume_1d', 'low_move_1d_diff', 'high_move_pct_1d_diff_pct_change', 'prev_close_minus_low_1d', 'open_minus_prev_close_1d_pct_change', 'prev1_target_HL5_ewm_mean', 'prev1_target_rolling_mean', 'prev4_move_1d', 'move_1d_diff_pct_change', 'high_minus_prev_high_1d_diff_pct_change', 'low_minus_prev_high_1d_diff', 'low_move_pct_1d_diff_pct_change', 'prev_close_pct_chg_1d', 'high_minus_close_1d_pct_change', 'high_move_pct_1d', 'high_move_1d_diff', 'move_1d_diff', 'open_

In [5]:
### Train model ###

model_dict = RunModel(X_test=X_transformed,
                      features=final_features,
                      X_train=X_train_transformed.tail(10000),
                      y_train=y_train.tail(10000),
                      algorithm=ALGORITHM,
                      eval_set=[(X_val_transformed, y_val)],
                      df_full=df_numerai,
                      **RUN_MODEL_PARAMS).run_everything()

try:
    pred_colname = RUN_MODEL_PARAMS['prediction_colname']

except:
    pred_colname = 'prediction'


Running XGBRegressor...


Not using an eval_set for XGBRegressor...


XGBRegressor training done!

Returning df_full to df_pred in model_dict


In [6]:
########################
###### evaluation ######
########################

### run numerai analytics ###

importances = pd.DataFrame({(f, imp) for f, imp in zip(X_train_transformed.columns, model_dict['model'].feature_importances_)})\
                .rename(columns={0: 'feature', 1: 'importance'})\
                .sort_values(by='importance', ascending=False)

### corr_coefs for train / val / test ###

train_era_scores = model_dict['df_pred'][model_dict['df_pred'][SPLIT_COLNAME].str.startswith('train')]\
                    .groupby(DATE_COL)\
                    .apply(calc_coef, TARGET, pred_colname)

val_era_scores = model_dict['df_pred'][model_dict['df_pred'][SPLIT_COLNAME].str.startswith('val')]\
                    .groupby(DATE_COL)\
                    .apply(calc_coef, TARGET, pred_colname)
test_era_scores = model_dict['df_pred'][model_dict['df_pred'][SPLIT_COLNAME].str.startswith('test')]\
                    .groupby(DATE_COL)\
                    .apply(calc_coef, TARGET, pred_colname)

In [13]:
### plot the coef scores / print the hit rates ###

train_era_scores = pd.DataFrame(train_era_scores, columns=['era_score']).assign(era='train')
val_era_scores = pd.DataFrame(val_era_scores, columns=['era_score']).assign(era='val')
test_era_scores = pd.DataFrame(test_era_scores, columns=['era_score']).assign(era='test')
era_scores = pd.concat([train_era_scores, val_era_scores, test_era_scores])

fig = px.line(era_scores.reset_index(), x="date", y="era_score", line_group='era', color='era')
fig.show()

### save model

In [79]:
if SAVE_MODEL:
    dill.dump(model_dict['model'], open(MODEL_OUTPATH + \
                                        type(model_dict['model']).__name__ + '_' + \
                                        str(datetime.datetime.today()\
                                            .replace(second=0, microsecond=0))\
                                            .replace(' ', '_')\
                                            .replace(':', '_') + '.pkl',\
                                            'wb'))

### submit model sandbox

In [45]:
df_val = pd.read_csv(napi.download_validation_data())
df_val = df_val[df_val['data_type'] == 'validation'].rename(columns={'bloomberg_ticker': 'ticker'})
print(df_val.shape)
df_val.head(2)

(2161626, 4)


,ticker,friday_date,data_type,target
2148958,000060 KS,20130104,validation,1.00000
2148959,000080 KS,20130104,validation,0.75000


In [41]:
df_to_submit = model_dict['df_pred'][\
                                     model_dict['df_pred']['data_type'] == 'validation'][\
                                                                                         ['bloomberg_ticker', 'friday_date', 'data_type', 'target', pred_colname]]\
                                                                                            .rename(columns={pred_colname: 'signal',
                                                                                                             'bloomberg_ticker': 'ticker'})
df_to_submit = df_to_submit[['ticker', 'signal', 'friday_date', 'data_type']]
print(df_to_submit.shape)
df_to_submit.head(2)

(2156196, 4)


,ticker,signal,friday_date,data_type
2148958,000060 KS,0.47222,20130104.00000,validation
2148959,000080 KS,0.58933,20130104.00000,validation


In [46]:
df_to_submit = pd.merge(df_to_submit,
                        df_val[['friday_date', 'ticker']], on=['friday_date', 'ticker'])
print(df_to_submit.shape)
df_to_submit.head(2)

(2156196, 4)


,ticker,signal,friday_date,data_type
0,000060 KS,0.47222,20130104.00000,validation
1,000080 KS,0.58933,20130104.00000,validation


'852fd5ba-3e09-4d50-b89a-4b91485aef7c'

In [52]:
df_to_submit.tail(2)

,ticker,signal,friday_date,data_type
2156194,ZYXI US,0.52891,20210402.00000,validation
2156195,ZZZ CN,0.51358,20210402.00000,validation


In [57]:
df_to_submit.to_csv('/media/melgazar9/HDD_10TB/trading/data/numerai/submissions/df_to_submit_example.csv')

In [58]:
# test submition
napi.upload_predictions('/media/melgazar9/HDD_10TB/trading/data/numerai/submissions/df_to_submit_example.csv',
                        model_id=list(napi.get_models().values())[0])

2021-04-24 12:39:21,783 INFO numerapi.base_api: uploading predictions...
2021-04-24 12:40:42,062 ERROR numerapi.base_api: headers must be signal, and bloomberg_ticker, cusip or sedol


ValueError: headers must be signal, and bloomberg_ticker, cusip or sedol